# Create WRF intermediate files

In [286]:
import os, glob, sys
import xarray as xr
import numpy as np
import pandas as pd

In [287]:
base_dir            = '/Users/doan/Google Drive/My Drive/share/2024/PGW_DS/'
run_id              = 'tok_test_dom'
src_dir             = base_dir + "/Run_WRF/"+run_id+"/control"
dst_dir             = base_dir + "/Run_WRF/"+run_id+"/D_DS"
wrf_inter_prefix    = 'ERA'

In [99]:
st_date, en_date = "2023-01-01 00", "2023-01-01 18"

In [127]:
da = xr.open_dataset('/Volumes/work/WRF_program/era5/era5_ungrib_pressure_levels_20230101.nc')
ds = xr.open_dataset('/Volumes/work/WRF_program/era5/era5_ungrib_surface_levels_20230101.nc')

In [150]:
varx      = {'TT':'t', 'UU':'u', 'RH':'r', 'VV':'v', 'GHT':'z'}
varx_s    = {'PSFC':'sp', 'PMSL':'msl', 'SKINTEMP':'skt'}
varx_soil = {'SM':'swvl', 'ST':'stl'}

In [261]:
import pywinter.winter as pyw

In [279]:
for time in pd.date_range(st_date,en_date, freq='6h')[:2]:
    print(time)
    date = time.strftime('%Y-%m-%d_%H')
    print(date)
    
    slat, slon = da.latitude.values[0], da.longitude.values[0]
    dlat, dlon = da.latitude.values[1] - da.latitude.values[0], da.longitude.values[1] - da.longitude.values[0]
    winter_geo = pyw.Geo0(slat,slon,dlat,dlon)

    total_fields = []
    
    level = da.level.values[::-1]*100
    
    for v in ['TT', 'UU', 'RH', 'VV', 'GHT']:
        values1 = da[varx[v]].sel(time=time)[::-1].values
        if v == 'GHT': values1 = values1 / 9.8
        winter = pyw.V3dp(v,values1,level)
        total_fields.append(winter)

    for v in [ 'PSFC', 'PMSL', 'SKINTEMP']:
        values1_s = ds[varx_s[v]].sel(time=time).values
        winter = pyw.V2d(v,values1_s)
        total_fields.append(winter)

    
    for v in ['SM', 'ST'][:]: # soil variables
        values1_so = np.array([ ds[varx_soil[v]+str(i)].sel(time=time).values for i in range(1,5)])
        sl_layer = ['000007', '007028', '028100', '100289'] 
        winter = pyw.Vsl(v,values1_so,sl_layer)
        total_fields.append(winter)
        
    pyw.cinter('ERA5AS',date, winter_geo, total_fields, dst_dir)
    

2023-01-01 00:00:00
2023-01-01_00
ERA5AS:2023-01-01_00
2023-01-01 06:00:00
2023-01-01_06
ERA5AS:2023-01-01_06


### Run WRF

In [281]:
import subprocess
base_dir            = '/Users/doan/Google Drive/My Drive/share/2024/PGW_DS/'
run_dir             = base_dir + "/Run_WRF/tok_test_dom/D_DS"

In [282]:
for f in glob.glob(run_dir+"/wrfout*"): os.remove(f)
for f in glob.glob(run_dir+"/wrfin*"): os.remove(f)
for f in glob.glob(run_dir+"/wrfbdy*"): os.remove(f)
for f in glob.glob(run_dir+"/met_em*"): os.remove(f)
#for f in glob.glob(run_dir+"/ERA*"): os.remove(f)

In [283]:
subprocess.run(['./metgrid.exe'], cwd=run_dir)

Processing domain 1 of 1
 Processing 2023-01-01_00
    ERA5AS
 Processing 2023-01-01_06
    ERA5AS
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!  Successful completion of metgrid.  !
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Note: The following floating-point exceptions are signalling: IEEE_OVERFLOW_FLAG


CompletedProcess(args=['./metgrid.exe'], returncode=0)

In [284]:
subprocess.run(['./real.exe'], cwd=run_dir)

 starting wrf task            0  of            1


CompletedProcess(args=['./real.exe'], returncode=0)

In [285]:
command = ['mpirun', '-np', '4', './wrf.exe']
subprocess.run(command, cwd=run_dir, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)

CompletedProcess(args=['mpirun', '-np', '4', './wrf.exe'], returncode=0)

### For reference

In [136]:
import pywinter.winter as pyw
for time in pd.date_range(st_date,en_date, freq='3H')[:3]:
    print(time)
    date = time.strftime('%Y-%m-%d_%H')
    print(date)
    prefix1 = 'ERA5A'
    infile = src_dir+'/'+prefix1+':' + date
    interfile = pyw.rinter(infile)
    infile = src_dir+'/ERA5S:' + date
    interfile_s = pyw.rinter(infile)
    
    var = list(interfile.keys())
    
    x = interfile[var[0]]
    slat, slon = x.geoinfo['STARTLAT'], x.geoinfo['STARTLON']
    dlat, dlon = x.geoinfo['DELTALAT'], x.geoinfo['DELTALON']
    
    winter_geo = pyw.Geo0(slat,slon,dlat,dlon)

    total_fields = []

    for v in var:
        x = interfile[v]
        values = x.val
        winter = pyw.V3dp(v,values,x.level)
        total_fields.append(winter)
        print(values.max())

    for v in list(interfile_s.keys()):
        
        x = interfile_s[v]
        values = x.val
        
        #=======================
        # Here to write intermediate
        if v in ['SM', 'ST']: # soil variables
            sl_layer = x.level
            winter = pyw.Vsl(v,values,sl_layer)
        elif v in [
                   'PSFC', 
                   #'LANDSEA', # no need (optional)
                   #'SNOW', # no need (optional)
                   # 'SST', # no need (optional)
                    #'SEAICE', # no need (optional)
                    'PMSL', 
                    'SKINTEMP'
                  ]:
            winter = pyw.V2d(v,values,x.general['DESC'],x.general['UNITS'], x.general['XLVL'])
        else: continue
        print(v)
        total_fields.append(winter)
        
    pyw.cinter('ERA5AS',date, winter_geo, total_fields, dst_dir)
    


2023-01-01 00:00:00
2023-01-01_00
292.818115234375
93.95826721191406
105.40580749511719
18.516143798828125
47485.171875
PSFC
ST
SM
PMSL
SKINTEMP
ERA5AS:2023-01-01_00
2023-01-01 03:00:00
2023-01-01_03
293.077392578125
92.86693572998047
101.76820373535156
16.394668579101562
47521.9296875
PSFC
ST
SM
PMSL
SKINTEMP
ERA5AS:2023-01-01_03
2023-01-01 06:00:00
2023-01-01_06
293.684326171875
92.63107299804688
103.9903793334961
20.416107177734375
47598.84375
PSFC
ST
SM
PMSL
SKINTEMP
ERA5AS:2023-01-01_06


/var/folders/y7/r3_l1bxn4sv6mkmyfgpt9n880000gn/T/ipykernel_67563/316169199.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for time in pd.date_range(st_date,en_date, freq='3H')[:3]:
